# multiprocessingによる並列処理

ハードウェアの確認
* コア数の確認 
```
sysctl -n hw.physicalcpu_max 
>> 2
```
* スレッド数の確認
```
sysctl -n hw.logicalcpu_max
>> 4 
```
手元のPCでは2コア4スレッド

In [1]:
import multiprocessing as mp
import os
import numpy
from multiprocessing import Pool, Process
import time

## Poolを使ってみる
Poolはサブプロセス群に入力データを分配 (データ並列) して関数を並列実行するのに便利
Retun簡単でいい。
これがベストプラクティスぽい

実行方法の違いを確認

In [2]:
print(os.cpu_count())

def f(x):
    print('input', x)
    print('return', x)
    return x*x
    
if __name__ == '__main__':
    #1コア1スレッドでの実行
    start = time.time()
    for i in range(8):
        _ = f(i)
    end = time.time()
    print("elapsed time: {} ".format(end - start))
    #4スレッドでの実行
    with Pool(2) as p: #デフォルトでos.cpu_count()と同じ数のプロセス
        start = time.time()
        p.map(f, list(range(8)))
        end = time.time()
        print("elapsed time: {} ".format(end - start))

4
input 0
return 0
input 1
return 1
input 2
return 2
input 3
return 3
input 4
return 4
input 5
return 5
input 6
return 6
input 7
return 7
elapsed time: 0.007734060287475586 
input 0
input 1
return 0
return 1
input 2
return 2
input 3
return 3
input 4
return 4
input 5
input 6
return 5
return 6
input 7
return 7
elapsed time: 0.10392498970031738 


## Processを使ってみる
Processにより各プロセスを制御することができる
parent process id：この実行ファイルのid

In [3]:
def info(title):
    #print(title)
    #print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)
    return name

def cal(x):
    result = []
    for i in x:
        result.append(i*i)
    print(result)
    return result
if __name__ == '__main__':
    X = list(range(8))
    #info('main line')
    #print('main')
    start = time.time()
    cal(X)
    end = time.time()
    print(end - start)
    print('この実行ファイルのprocess ID:', os.getpid())
    start = time.time()
    #subprocessの生成  
    
    interval = int(len(X)/os.cpu_count())
    p_list = []
    for i in range(os.cpu_count()):
        x = X[i*interval:(i+1)*interval]
        p_list.append(Process(target=cal, args=(x,)))
    for p_i in p_list :
        p_i.start()
    end = time.time()
    print(end - start)

[0, 1, 4, 9, 16, 25, 36, 49]
0.00019478797912597656
この実行ファイルのprocess ID: 7011
[0, 1]
[4, 9]
[16, 25]
[36, 49]
0.07054710388183594


In [46]:
from multiprocessing import Process
import numpy as np
def info(title):
    #print(title)
    #print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)
    return name

def cal(stnd):
    result = 0
    for i in range(stnd[0], stnd[1]):
        result += i
    return result

if __name__ == '__main__':
    print(mp.cpu_count())
    N=10000000
    X = np.arange(N)
    print("{}Mb".format(X.nbytes/1024/1024))
    interval = int(N/2)
    
    start = time.time()
    print(cal([0, N]))
    end = time.time()
    print("elapsed time: {} ".format(end - start))
    
    print('この実行ファイルのprocess ID:', os.getpid())
    with Pool(2) as p: #デフォルトでos.cpu_count()と同じ数のプロセス
        start = time.time()
        output = p.map(cal,  [ [i*interval, (i+1)*interval] for i in range(2)])
        print(sum(output))
        end = time.time()
        print("elapsed time: {} ".format(end - start))
    #print(output)

4
76.2939453125Mb
49999995000000
elapsed time: 1.3334672451019287 
この実行ファイルのprocess ID: 7061
49999995000000
elapsed time: 0.7098281383514404 
